## Lazy Sentiment Analysis Network

Using the same model as before lets see how it performs on a totally different type of problem.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from collections import Counter

tf.logging.set_verbosity(tf.logging.ERROR)

Read IMDB movie reviews dataset.

In [ ]:
reviews = pd.read_csv('./data/imdb/reviews.txt', header=None)
labels = pd.read_csv('./data/imdb/labels_ohe.csv', header=None)

In [ ]:
print(len(reviews))
print(len(labels))

In [ ]:
reviews.head()

In [ ]:
reviews[0][3]

In [ ]:
labels.head()

In [ ]:
labels.apply(pd.value_counts)

Count the number of unique words in the entire dataset.

In [ ]:
total_counts = Counter()
for i,row in reviews.iterrows():
    total_counts.update(row[0].split(' '))

print("Total number of unique words in data set: ", len(total_counts))

Place all the unique words in the dataset in a list, sorted by most frequent word first. This is the vocabulary.

In [ ]:
vocab = sorted(total_counts, key=total_counts.get, reverse=True)[:10000]
print(vocab[0:50])

Inspect the number of occurences of one word.

In [ ]:
print(vocab[77], ': ', total_counts[vocab[77]])

Assign an index to each word in the vocabulary.

In [ ]:
word2idx = {word: i for i, word in enumerate(vocab)} #dictionary comprehension

In [ ]:
word2idx

This function converts text to a vector that indicates the occurance of a word in the vocabulary. However, it does not count the number of occurances.

In [ ]:
def text_to_vector(text):
    word_vector = np.zeros(len(vocab), dtype=np.int_)
    for word in text.split(' '):
        idx = word2idx.get(word,None)
        if idx is None:
            continue
        else:
            word_vector[idx] = 1 # was += 1
    return np.array(word_vector)

In [ ]:
text_to_vector('There were lots of good movies and stars this year')[:65]

Let's convert the dataset 25000 text reviews to 25000 word vectors.

In [ ]:
word_vectors = np.zeros((len(reviews), len(vocab)), dtype=np.int_)
for ii, (_, text) in enumerate(reviews.iterrows()):
    word_vectors[ii] = text_to_vector(text[0])

In [ ]:
# Printing out the first 25 elements of the first 5 word vectors
print(word_vectors[:5, :25])
print(reviews[:5])

In [ ]:
word_vectors.shape

In [ ]:
number_of_records = len(labels)
shuffle = np.arange(number_of_records)
np.random.shuffle(shuffle)
test_fraction = 0.9

#making a train / test split
train_split, test_split = shuffle[:int(number_of_records*test_fraction)], shuffle[int(number_of_records*test_fraction):]
trainX, trainY = word_vectors[train_split,:], labels.values[train_split,:]
testX, testY = word_vectors[test_split,:], labels.values[test_split]

print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)
type(testX[0][0])

In [ ]:
#making a quick batch system
def get_next_batch(batch_size,i):
    return trainX[(i*batch_size):((i+1)*batch_size)].astype('float32'),trainY[(i*batch_size):((i+1)*batch_size)].astype('float32')

# Testing
batch_x, batch_y = get_next_batch(100,3)
print(batch_x[0:5])
print(batch_y[0:5])
print(batch_y.shape)

In [ ]:
# Training Hyper Parameters 
learning_rate = 0.001 
training_epochs = 20
batch_size = 100
display_step = 1  # for how often to print out our results
model_path = "./models_sentiment/model_sentiment.ckpt"

# Network Parameters
n_input = 10000 # we now have 10k vectors of our words
n_hidden_1 = 384 # 1st layer number of neurons
n_hidden_2 = 100 # 2nd layer number of neurons
n_classes = 2 # 2 classes for predicting positive or negative

In [ ]:
# Session Configuration
config = tf.ConfigProto()
config.gpu_options.allow_growth=True   # don't allow session to take up all the GPU memory

# The graph
tf.reset_default_graph()
sess = tf.Session()

# tf Graph input
x = tf.placeholder("float", [None, n_input],name='X_Input')
y = tf.placeholder("float", [None, n_classes],name='Y_Input')

In [ ]:
# Create model
def multilayer_perceptron(x, weights, biases):
    
    # Hidden layer 01 with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])  # adding (x + w1 + bias1)
    layer_1 = tf.nn.relu(layer_1, name='Layer1_Relu') #activation
    
    # Hidden layer 02 with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2, name='Layer2_Relu')
    
    # Logits layer with linear activation
    logits_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    #logits_layer = tf.nn.softmax(logits_layer)
    return logits_layer

In [ ]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1],stddev = 0.1)),
    'h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2],stddev = 0.1)),
    'out': tf.Variable(tf.truncated_normal([n_hidden_2, n_classes],stddev = 0.1))
}
biases = {
    'b1': tf.Variable(tf.ones([n_hidden_1])/10),
    'b2': tf.Variable(tf.ones([n_hidden_2])/10),
    'out': tf.Variable(tf.ones([n_classes])/10)
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)


In [ ]:
# Define loss and optimizer

# this is were we compute error against the correct results
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))*100

# optimizer made to change weights and biases to optimize cost
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss) 
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

# Initializing the variables
init = tf.global_variables_initializer()

In [ ]:
# 'Saver' op to save and restore all the variables
saver = tf.train.Saver()

In [ ]:
summary_y = tf.summary.scalar('output', y)
file_writer = tf.summary.FileWriter('log_simple_sentiment')

In [ ]:
%%time
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(trainX.shape[0]/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = get_next_batch(100,i)
            #print(batch_x[0])
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, loss], feed_dict={x: batch_x, y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print ("Epoch:", '%04d' % (epoch+1), "Loss =", \
                "{:.9f}".format(avg_cost))
    print ("Training Finished!")
    
    # Save model weights to disk
    save_path = saver.save(sess, model_path)
    print ("Model saved in file: %s" % save_path)

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    
    # Restore model weights from previously saved model
    load_path = saver.restore(sess, model_path)
    print ("Model restored from file: %s" % save_path)
    
    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print ("Accuracy:", accuracy.eval({x: testX, y: testY}))

In [ ]:
def predict_text(text_string):
    textVec= text_to_vector(text_string)
    with tf.Session() as sess:
        sess.run(init)
        # Restore model weights from previously saved model
        load_path = saver.restore(sess, model_path)
        # Predict model 1 image batch size = 1
        vector = sess.run(pred, feed_dict={x: [textVec]})
        #print(vector[0])
        pred_label = sess.run(tf.argmax(vector[0],0))
        #print the label
        if pred_label == 1: 
            print('Positive')
        else:
            print('Negative')

In [ ]:
predict_text("lion is a great movie to watch this year")

In [ ]:
predict_text("this was worst experience in a long")